In [1]:
import pandas
import numpy
import datetime
import synapseutils
import synapseclient
from synapseclient import Entity, Project, Column, Team, Wiki

In [2]:
def synapseLogin():
    """
    First tries to login to synapse by finding the local auth key cached on user's computing platform, if not found,
    prompts the user to provide their synapse user name and password, then caches the auth key on their computing
    platform.

    :return:
    """
    try:
        syn = synapseclient.login()
    except Exception as e:
        print('Please provide your synapse username/email and password (You will only be prompted once)')
        username = input("Username: ")
        password = getpass.getpass(("Password for " + username + ": ").encode('utf-8'))
        syn = synapseclient.login(email=username, password=password, rememberMe=True)

    return syn

In [3]:
synapseLogin()

Welcome, Xengie Doan!



In [4]:
def getdf(syn, id):
    """

    :param syn:
    :param id:
    :return:
    """
    df = syn.tableQuery('select * from {id}'.format(id=id)).asDataFrame()
    return df

In [5]:
"""
Create a master matrix/table for consortium metrics.

:param args:
:param syn:
:return:
"""
# project and publication attributes
p_atr = ['projectName',
         'id',
         'fundingAgency',
         'citation',
         'doi',
         'tumorType',
         'diseaseFocus']

### from table syn16787123
p_view_atr = ['projectName',
              'id',
              'projectFileviewId',
              'projectStatus',
              'dataStatus',
              'fundingAgency',
              'summary',
              'summarySource',
              'projectLeads',
              'institutions',
              'tumorType',
              'diseaseFocus']


# file view attributes
f_atr = ['id',
        'name',
        'projectId',
        'assay',
        'consortium',
        'dataSubtype',
        'dataType',
        'diagnosis',
        'tumorType',
        'fileFormat',
        'fundingAgency',
        'individualID',
        'nf1Genotype',
        'nf2Genotype',
        'species',
        'resourceType',
        'isCellLine',
        'isMultiSpecimen',
        'isMultiIndividual',
        'studyId',
        'studyName',
        'benefactorId',
        'specimenID',
        'sex'
        'age',
        'readPair',
        'createdOn',
        'modifiedOn']

# csbc project info integration 
csbc_atr = ["projectId",
            "name_project",
            "consortium",
            "institution",
            "grantNumber",
            "grantType",
            "teamMembersProfileId",
            "teamProfileId",
            "createdOn_project",
            "modifiedOn_project",
            "publication_count",
            "publication_geodata_produced",
            "fileId","name_file",
            "createdOn_file",
            "modifiedOn_file",
            "age",
            "analysisType",
            "assay",
            "cellLine",
            "cellSubType",
            "cellType",
            "compoundDose",
            "compoundName",
            "concreteType",
            "dataSubtype",
            "dataType",
            "diagnosis",
            "diseaseSubtype",
            "dnaAlignmentMethod",
            "experimentalCondition",
            "experimentalTimePoint",
            "fileFormat",
            "fundingAgency",
            "individualID",
            "individualIdSource",
            "inputDataType",
            "isCellLine",
            "isPrimaryCell",
            "isStranded",
            "libraryPrep",
            "modelSystem",
            "organ",
            "outputDataType",
            "peakCallingMethod",
            "platform",
            "readLength",
            "resourceType",
            "rnaAlignmentMethod",
            "runType",
            "scriptLanguageVersion",
            "sex","softwareAuthor",
            "softwareLanguage",
            "softwareRepository",
            "softwareRepositoryType",
            "softwareType",
            "species",
            "specimenID",
            "study",
            "tissue",
            "transcriptQuantificationMethod",
            "transplantationDonorSpecies",
            "transplantationDonorTissue",
            "transplantationRecipientTissue",
            "transplantationType",
            "tumorType"]

In [6]:
# merging all the things
# 0 publications view syn16857542
# 1 project table  syn16787123
# 2 all portal - files syn16858331
# 3 tools syn9898965
views = ['syn16857542', 'syn16787123', 'syn16858331', 'syn16859448']

dfs = [getdf(synapseclient.login(), synid) for synid in views]
[d.reset_index(inplace=True, drop=True) for d in dfs]

Welcome, Xengie Doan!

Welcome, Xengie Doan!

Welcome, Xengie Doan!

Welcome, Xengie Doan!



[None, None, None, None]

In [7]:
# Project attributes
# change columns to represent project attributes and unify key name to be projectId
dfs[0].rename(index=str, columns={"id": "projectId", "name" : "projectName"}, inplace=True)
dfs[1].rename(index=str, columns={"id": "projectId", "name": "projectName"}, inplace=True)

In [11]:
dfs[2].rename(index=str, columns={"id": "fileId", "name": "name_file", "createdOn": "createdOn_file",
                                  "modifiedOn": "modifiedOn_file", "modifiedBy": "modifiedBy_file"}, inplace=True)
dfs[3].rename(index=str, columns={"id": "fileId", "name": "name_file", "createdOn": "createdOn_file",
                                  "modifiedOn": "modifiedOn_file", "modifiedBy": "modifiedBy_file", "study" :"projectId"}, inplace=True)


In [8]:
# take out non CTF funded projects
dfs[0] = dfs[0][~dfs[0].fundingAgency.isin(['NTAP', 'NIH-NCI'])]
dfs[1] = dfs[1][~dfs[1].fundingAgency.isin(['NTAP', 'NIH-NCI'])]
dfs[2] = dfs[2][~dfs[2].fundingAgency.isin(['NTAP', 'NIH-NCI'])]
dfs[3] = dfs[3][~dfs[3].fundingAgency.isin(['NTAP', 'NIH-NCI'])]

In [29]:
pandas.set_option('display.max_columns', 500)
dfs[1]


,projectName,projectId,projectFileviewId,projectStatus,dataStatus,fundingAgency,summary,summarySource,projectLeads,institutions,tumorType,diseaseFocus,featured,consortium,publication_count,publication_geodata_produced
1,Synodos NF2,syn2343195,syn11614200,Completed,Published,CTF,Synodos for NF2 is a first-of-its-kind neurofi...,https://www.synapse.org/#!Synapse:syn2343195/w...,Jaishri Blakeley | Wade Clapp | James Gusella ...,Johns Hopkins Medical Institute | Indiana Univ...,Schwannoma | Meningioma,Neurofibromatosis 2,yes,NaN,0,0
22,Cutaneous Neurofibroma Data Resource,syn4984604,syn11614202,Completed,Published,CTF,"In 2013, CTF began a neurofibromatosis biobank...",https://www.synapse.org/#!Synapse:syn4984604/w...,Salvatore La Rosa | Annette Bakker | Justin G...,Children's Tumor Foundation | Sage Bionetworks...,Cutaneous Neurofibroma,Neurofibromatosis 1,NaN,NaN,0,0
23,Synodos NF1 Preclinical Models (Iowa Sanford E...,syn5610446,syn15672867,Active,Under Embargo,CTF,The aim of Synodos for NF1 is to find solution...,https://www.synapse.org/#!Synapse:syn5610418/w...,Jill Weimer,University of Iowa | University of Arizona | E...,Plexiform Neurofibroma,Neurofibromatosis 1,NaN,NaN,0,0
24,Synodos NF1 Preclinical Models (Minnesota CCHM...,syn5610425,syn15672871,Active,Under Embargo,CTF,The aim of Synodos for NF1 is to find solution...,https://www.synapse.org/#!Synapse:syn5610418/w...,David Largaespada,University of Minnesota | Cincinnati Children'...,Plexiform Neurofibroma,Neurofibromatosis 1,NaN,NaN,2,0
29,Synodos NF1 Low Grade Glioma,syn5698493,syn11614205,Active,Under Embargo,CTF,Low-grade gliomas (LGGs) are brain tumors that...,https://www.synapse.org/#!Synapse:syn5698493/w...,David Gutmann | Michael Fisher,Washington University | Children's Hospital of...,Low Grade Glioma,Neurofibromatosis 1,NaN,NaN,2,0
30,Columbia University NF1 Glioma Project,syn6633069,syn11614207,Active,Under Embargo,CTF,This study aims to obtain the genomic landscap...,https://www.synapse.org/#!Synapse:syn6135075/w...,Antonio Iaverone,Columbia University | University of Sannio,Low Grade Glioma,Neurofibromatosis 1,NaN,NaN,2,0
36,Evotec Drug Target Research,syn7239560,syn11614208,Completed,Published,CTF,A drug-target database was built by Evotec usi...,Robert Allaway,Tim James,Evotec,NaN,Multiple,NaN,NaN,0,0
43,Synodos Schwannomatosis,syn9727752,syn11614209,Active,Under Embargo,CTF,Synodos for Schwannomatosis is a two-year init...,https://www.synapse.org/#!Synapse:syn9727752,Gelareh Zadeh | Laura Papi,University Health Network of Toronto | Univers...,Schwannoma,Schwannomatosis,NaN,NaN,3,0
52,Drug-Target Explorer,syn11672851,syn15665015,Active,Published,CTF,Phenotypic high-throughput screens are often u...,https://www.synapse.org/#!Synapse:syn11672851/...,Robert Allaway | Sara Gosline,Sage Bionetworks,NaN,Multiple,NaN,NaN,0,0
53,NF Cell Line Compound Screens,syn11817821,syn11818313,Completed,Partially Released,CTF,This project created the first college course ...,https://www.synapse.org/#!Synapse:syn11817821,Jeffrey Field,University of Pennsylvania,MPNST,Neurofibromatosis 1,NaN,NaN,0,0


In [10]:
dfs[2].studyName.unique() #there are 4 consoritums under the CTF funding agency

array(['Cutaneous Neurofibroma Data Resource',
       'Phenotypic screening of anti-NF2 drug candidates in merlin-deficient and normal mouse Schwann cells',
       'Synodos NF1 Preclinical Models (Iowa Sanford Exemplar)',
       'Synodos NF1 Preclinical Models (Minnesota CCHMC Recombinetics)',
       'Synodos NF1 LGG', nan, 'Synodos NF2',
       'Evotec Drug Target Research',
       'Columbia University NF1 Glioma Project',
       'Synodos NF1 Low Grade Glioma', 'Drug-Target Explorer',
       'NF Cell Line Compound Screens', 'Synodos Schwannomatosis',
       'Identifying preclinical predictors of clinical success in NF1'],
      dtype=object)

In [13]:
nf1 = ['Synodos NF1 Preclinical Models (Iowa Sanford Exemplar)',
       'Synodos NF1 Preclinical Models (Minnesota CCHMC Recombinetics)']
dfs[2].loc[dfs[2]['studyName'].isin(nf1)]

### dfs[2] has the files

,fileId,name_file,projectId,assay,consortium,dataSubtype,dataType,diagnosis,tumorType,fileFormat,fundingAgency,individualID,nf1Genotype,nf2Genotype,species,resourceType,isCellLine,isMultiSpecimen,isMultiIndividual,studyId,studyName,etag,benefactorId,specimenID,sex,age,readPair,createdOn_file,modifiedOn_file,parentId,reportMilestone,createdBy
877,syn5611533,Weimer_Meyerholz_NF1 Final Application.pdf,syn5610418,NaN,Synodos,report,NaN,Neurofibromatosis 1,NaN,pdf,CTF,NaN,NaN,NaN,NaN,report,NaN,NaN,NaN,syn5610446,Synodos NF1 Preclinical Models (Iowa Sanford E...,70c35333-7ab2-11e9-98fa-026b0a0ad230,syn5610446,NaN,NaN,NaN,NaN,1453941199154,1543522388961,syn5610446,NaN,1418096
878,syn5611534,Revised NF1 Synodos Application UMN.pdf,syn5610418,NaN,Synodos,report,NaN,Neurofibromatosis 1,NaN,pdf,CTF,NaN,NaN,NaN,NaN,report,NaN,NaN,NaN,syn5610425,Synodos NF1 Preclinical Models (Minnesota CCHM...,70c353cb-7ab2-11e9-98fa-026b0a0ad230,syn5610425,NaN,NaN,NaN,NaN,1453941294460,1543522389047,syn5610425,NaN,1418096
2248,syn6057319,Description of Pig Imaging Files.xlsx,syn5610418,MRI,Synodos,metadata,image,Neurofibromatosis 1,NaN,excel,CTF,NaN,NaN,+/+,NaN,experimentalData,NaN,NaN,NaN,syn5610425,Synodos NF1 Preclinical Models (Minnesota CCHM...,717d418e-7ab2-11e9-98fa-026b0a0ad230,syn5610425,NaN,NaN,NaN,NaN,1463158212275,1543522578144,syn6047677,NaN,3336397
2658,syn6129980,Pig1728_DICOM.7z,syn5610418,MRI,Synodos,raw,image,Neurofibromatosis 1,NaN,7z,CTF,NaN,+/-,+/+,Sus scrofa,experimentalData,False,False,False,syn5610425,Synodos NF1 Preclinical Models (Minnesota CCHM...,71eb9058-7ab2-11e9-98fa-026b0a0ad230,syn5610425,NaN,NaN,NaN,NaN,1464726116151,1543522641654,syn6047677,NaN,3336397
2659,syn6129999,Pig1734_DICOM.7z,syn5610418,MRI,Synodos,raw,image,Neurofibromatosis 1,NaN,7z,CTF,NaN,+/-,+/+,Sus scrofa,experimentalData,False,False,False,syn5610425,Synodos NF1 Preclinical Models (Minnesota CCHM...,71eb966c-7ab2-11e9-98fa-026b0a0ad230,syn5610425,NaN,NaN,NaN,NaN,1464726934389,1543522642387,syn6047677,NaN,3336397
3322,syn7253558,HSC1L_S27_R1_001.fastq.gz,syn5610418,rnaSeq,Synodos,raw,geneExpression,Neurofibromatosis 1,NaN,fastq,CTF,HSC1L,+/+,+/+,Human,experimentalData,True,False,False,syn5610425,Synodos NF1 Preclinical Models (Minnesota CCHM...,72ba5795-7ab2-11e9-98fa-026b0a0ad230,syn5610425,HSC1L_S27,NaN,NaN,NaN,1474296685190,1543522738656,syn5643678,NaN,3336397
3323,syn7253570,HSC1L_S27_R2_001.fastq.gz,syn5610418,rnaSeq,Synodos,raw,geneExpression,Neurofibromatosis 1,NaN,fastq,CTF,HSC1L,+/+,+/+,Human,experimentalData,True,False,False,syn5610425,Synodos NF1 Preclinical Models (Minnesota CCHM...,72ba5a94-7ab2-11e9-98fa-026b0a0ad230,syn5610425,HSC1L_S27,NaN,NaN,NaN,1474298311553,1543522739227,syn5643678,NaN,3336397
3324,syn7253585,N3_S26_R1_001.fastq.gz,syn5610418,rnaSeq,Synodos,raw,geneExpression,Neurofibromatosis 1,NaN,fastq,CTF,N3,+/+,+/+,Human,experimentalData,True,False,False,syn5610425,Synodos NF1 Preclinical Models (Minnesota CCHM...,72ba5e35-7ab2-11e9-98fa-026b0a0ad230,syn5610425,N3_S26,NaN,NaN,NaN,1474299911294,1543522739740,syn5643678,NaN,3336397
3325,syn7253596,N3_S26_R2_001.fastq.gz,syn5610418,rnaSeq,Synodos,raw,geneExpression,Neurofibromatosis 1,NaN,fastq,CTF,N3,+/+,+/+,Human,experimentalData,True,False,False,syn5610425,Synodos NF1 Preclinical Models (Minnesota CCHM...,72ba5f65-7ab2-11e9-98fa-026b0a0ad230,syn5610425,N3_S26,NaN,NaN,NaN,1474301196364,1543522739811,syn5643678,NaN,3336397
3326,syn7254141,N5_S29_R1_001.fastq.gz,syn5610418,rnaSeq,Synodos,raw,geneExpression,Neurofibromatosis 1,NaN,fastq,CTF,N5,-/-,+/+,Human,experimentalData,True,False,False,syn5610425,Synodos NF1 Preclinical Models (Minnesota CCHM...,72ba938d-7ab2-11e9-98fa-026b0a0ad230,syn5610425,N5_S29,NaN,NaN,NaN,1474325690247,1543522739879,syn5643678,NaN,3336397


In [14]:
# there are projects without publications
len(set(dfs[1].projectId.unique()) - set(dfs[0].projectId.unique()))


3

In [15]:
# Associate publications information to projects
project_info_df = pandas.merge(dfs[1], dfs[0], on=['projectId','projectName', 'fundingAgency', 'diseaseFocus', 'tumorType'], how='left')


In [16]:
project_info_df = project_info_df[
    [ 'projectName',
     'projectId',
     'projectFileviewId',
     'dataStatus',
     'fundingAgency',
     'projectLeads',
     'institutions',
     'tumorType',
     'diseaseFocus',
     'citation', 
     'doi']
]

In [17]:
publication_count = list(project_info_df.groupby(['projectId']))
dfs[1]['publication_count'] = [len(x[1]) if len(x[1]) != 1 else 0 for x in publication_count]

In [18]:
dfs[0] = dfs[0].astype(object).replace(numpy.nan, '')

In [19]:
dfs[1]['publication_geodata_produced'] = 0  ### don't have data location...run getPMIDDF or set to zero

In [20]:
dfs[3]

,softwareName,summary,softwareLink,featured,link,studyId,studyName,fundingAgency,contact,type,subtype,disease,manifestation,name_file
0,Drug-Target Explorer,The Drug Target Explorer helps identify small ...,https://shiny.synapse.org/users/rallaway/polyp...,NaN,https://shiny.synapse.org/users/rallaway/polyp...,syn11672851,Drug-Target Explorer,CTF,Robert Allaway,computational,visualization,Multiple,Multiple,Drug-Target Explorer
1,Synodos NF2 Data Explorer,The Synodos for NF2 study generated a broad ar...,https://www.synapse.org/#!Wiki:syn6140303/ENTI...,NaN,https://www.synapse.org/#!Wiki:syn6140303/ENTI...,syn2343195,Synodos NF2,CTF,Robert Allaway,computational,visualization,NF2,Schwannoma | Meningioma,Synodos NF2 Data Explorer
5,NF1 Variant Annotation Server,A webserver describing the entire set of nonsy...,https://isakovlab.shinyapps.io/NF1-VariantAnno...,NaN,https://isakovlab.shinyapps.io/NF1-VariantAnno...,NaN,NaN,NaN,Ofer Isakov | Shay Ben-Shachar,computational,prediction,NF1,Multiple,NF1 Variant Annotation Server
7,NF1-flox Mice,B6;129 mice with a floxed Nf1 allele to allow ...,https://frederick.cancer.gov/science/technolog...,NaN,https://frederick.cancer.gov/science/technolog...,NaN,NaN,NaN,Luis Parada,bench science,animal model,NF1,None,NF1-flox Mice
8,CTF Biobank,A biobank of donated tissue that is available ...,https://www.ctf.org/understanding-nf/ctf-biobank,NaN,https://www.ctf.org/understanding-nf/ctf-biobank,NaN,NaN,CTF,Pamela Knight,bench science,biobank,Multiple,Multiple,CTF Biobank
9,Nf1/Trp53 cis Mice,B6;129 mice with mutations in Nf1 and Trp53 (i...,https://www.jax.org/strain/008191,NaN,https://www.jax.org/strain/008191,NaN,NaN,NaN,Tyler Jacks,bench science,animal model,NF1,Astrocytoma | MPNST,Nf1/Trp53 cis Mice


In [50]:
# File attributes
# remove tools files (subset of all datafiles) from all datafiles
tools_files_id = list(set(dfs[2]["name_file"].unique()).intersection(set(dfs[3]["name_file"].unique())))

# # no files that are also tools for CTF
list(set(dfs[3]["name_file"].unique()).intersection( set(dfs[2]["name_file"].unique())))

[]

In [23]:
cols_to_add2 = dfs[3].columns.difference(dfs[2].columns)
cols_to_add3 = dfs[2].columns.difference(dfs[3].columns)
dfs[2] = pandas.concat([dfs[2], pandas.DataFrame(columns=cols_to_add2)])
dfs[3] = pandas.concat([dfs[3], pandas.DataFrame(columns=cols_to_add3)])


[WARNING] /Users/xdoan/anaconda2/envs/py3.5/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until

[WARNING] /Users/xdoan/anaconda2/envs/py3.5/lib/python3.5/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.



In [24]:
dfs[2].loc[dfs[2]['studyName'].isin(nf1)]

,age,assay,benefactorId,consortium,contact,createdBy,createdOn_file,dataSubtype,dataType,diagnosis,disease,etag,featured,fileFormat,fileId,fundingAgency,individualID,isCellLine,isMultiIndividual,isMultiSpecimen,link,manifestation,modifiedOn_file,name_file,nf1Genotype,nf2Genotype,parentId,projectId,readPair,reportMilestone,resourceType,sex,softwareLink,softwareName,species,specimenID,studyId,studyName,subtype,summary,tumorType,type
877,NaN,NaN,syn5610446,Synodos,NaN,1418096.0,1.453941e+12,report,NaN,Neurofibromatosis 1,NaN,70c35333-7ab2-11e9-98fa-026b0a0ad230,NaN,pdf,syn5611533,CTF,NaN,NaN,NaN,NaN,NaN,NaN,1.543522e+12,Weimer_Meyerholz_NF1 Final Application.pdf,NaN,NaN,syn5610446,syn5610418,NaN,NaN,report,NaN,NaN,NaN,NaN,NaN,syn5610446,Synodos NF1 Preclinical Models (Iowa Sanford E...,NaN,NaN,NaN,NaN
878,NaN,NaN,syn5610425,Synodos,NaN,1418096.0,1.453941e+12,report,NaN,Neurofibromatosis 1,NaN,70c353cb-7ab2-11e9-98fa-026b0a0ad230,NaN,pdf,syn5611534,CTF,NaN,NaN,NaN,NaN,NaN,NaN,1.543522e+12,Revised NF1 Synodos Application UMN.pdf,NaN,NaN,syn5610425,syn5610418,NaN,NaN,report,NaN,NaN,NaN,NaN,NaN,syn5610425,Synodos NF1 Preclinical Models (Minnesota CCHM...,NaN,NaN,NaN,NaN
2248,NaN,MRI,syn5610425,Synodos,NaN,3336397.0,1.463158e+12,metadata,image,Neurofibromatosis 1,NaN,717d418e-7ab2-11e9-98fa-026b0a0ad230,NaN,excel,syn6057319,CTF,NaN,NaN,NaN,NaN,NaN,NaN,1.543523e+12,Description of Pig Imaging Files.xlsx,NaN,+/+,syn6047677,syn5610418,NaN,NaN,experimentalData,NaN,NaN,NaN,NaN,NaN,syn5610425,Synodos NF1 Preclinical Models (Minnesota CCHM...,NaN,NaN,NaN,NaN
2658,NaN,MRI,syn5610425,Synodos,NaN,3336397.0,1.464726e+12,raw,image,Neurofibromatosis 1,NaN,71eb9058-7ab2-11e9-98fa-026b0a0ad230,NaN,7z,syn6129980,CTF,NaN,False,False,False,NaN,NaN,1.543523e+12,Pig1728_DICOM.7z,+/-,+/+,syn6047677,syn5610418,NaN,NaN,experimentalData,NaN,NaN,NaN,Sus scrofa,NaN,syn5610425,Synodos NF1 Preclinical Models (Minnesota CCHM...,NaN,NaN,NaN,NaN
2659,NaN,MRI,syn5610425,Synodos,NaN,3336397.0,1.464727e+12,raw,image,Neurofibromatosis 1,NaN,71eb966c-7ab2-11e9-98fa-026b0a0ad230,NaN,7z,syn6129999,CTF,NaN,False,False,False,NaN,NaN,1.543523e+12,Pig1734_DICOM.7z,+/-,+/+,syn6047677,syn5610418,NaN,NaN,experimentalData,NaN,NaN,NaN,Sus scrofa,NaN,syn5610425,Synodos NF1 Preclinical Models (Minnesota CCHM...,NaN,NaN,NaN,NaN
3322,NaN,rnaSeq,syn5610425,Synodos,NaN,3336397.0,1.474297e+12,raw,geneExpression,Neurofibromatosis 1,NaN,72ba5795-7ab2-11e9-98fa-026b0a0ad230,NaN,fastq,syn7253558,CTF,HSC1L,True,False,False,NaN,NaN,1.543523e+12,HSC1L_S27_R1_001.fastq.gz,+/+,+/+,syn5643678,syn5610418,NaN,NaN,experimentalData,NaN,NaN,NaN,Human,HSC1L_S27,syn5610425,Synodos NF1 Preclinical Models (Minnesota CCHM...,NaN,NaN,NaN,NaN
3323,NaN,rnaSeq,syn5610425,Synodos,NaN,3336397.0,1.474298e+12,raw,geneExpression,Neurofibromatosis 1,NaN,72ba5a94-7ab2-11e9-98fa-026b0a0ad230,NaN,fastq,syn7253570,CTF,HSC1L,True,False,False,NaN,NaN,1.543523e+12,HSC1L_S27_R2_001.fastq.gz,+/+,+/+,syn5643678,syn5610418,NaN,NaN,experimentalData,NaN,NaN,NaN,Human,HSC1L_S27,syn5610425,Synodos NF1 Preclinical Models (Minnesota CCHM...,NaN,NaN,NaN,NaN
3324,NaN,rnaSeq,syn5610425,Synodos,NaN,3336397.0,1.474300e+12,raw,geneExpression,Neurofibromatosis 1,NaN,72ba5e35-7ab2-11e9-98fa-026b0a0ad230,NaN,fastq,syn7253585,CTF,N3,True,False,False,NaN,NaN,1.543523e+12,N3_S26_R1_001.fastq.gz,+/+,+/+,syn5643678,syn5610418,NaN,NaN,experimentalData,NaN,NaN,NaN,Human,N3_S26,syn5610425,Synodos NF1 Preclinical Models (Minnesota CCHM...,NaN,NaN,NaN,NaN
3325,NaN,rnaSeq,syn5610425,Synodos,NaN,3336397.0,1.474301e+12,raw,geneExpression,Neurofibromatosis 1,NaN,72ba5f65-7ab2-11e9-98fa-026b0a0ad230,NaN,fastq,syn7253596,CTF,N3,True,False,False,NaN,NaN,1.543523e+12,N3_S26_R2_001.fastq.gz,+/+,+/+,syn5643678,syn5610418,NaN,NaN,experimentalData,NaN,NaN,NaN,Human,N3_S26,syn5610425,Synodos NF1 Preclinical Models (Minnesota CCHM...,NaN,NaN,NaN,NaN
3326,NaN,rnaSeq,syn5610425,Synodos,NaN,3336397.0,1.474326e+12,raw,geneExpression,Neurofibromatosis 1,NaN,72ba938d-7ab2-

In [33]:
### edit synodos nf1 preclinical studyId to projectId (both nested in the same projectId but have own files)
dfs[2].loc[dfs[2]['studyName'].isin(nf1), 'projectId'] = dfs[2].loc[dfs[2]['studyName'].isin(nf1), 'studyId']

In [37]:
# concat files and tools to get all the files information data frame
file_info_df = pandas.concat([dfs[3], dfs[2]], sort = False)

In [38]:
final_df = pandas.merge( dfs[1], file_info_df, on= ['projectId'], how='left')

In [39]:
final_df.loc[final_df['studyName'].isin(nf1)]

,projectName,projectId,projectFileviewId,projectStatus,dataStatus,fundingAgency_x,summary_x,summarySource,projectLeads,institutions,tumorType_x,diseaseFocus,featured_x,consortium_x,publication_count,publication_geodata_produced,age,assay,benefactorId,consortium_y,contact,createdBy,createdOn_file,dataSubtype,dataType,diagnosis,disease,etag,featured_y,fileFormat,fileId,fundingAgency_y,individualID,isCellLine,isMultiIndividual,isMultiSpecimen,link,manifestation,modifiedOn_file,name_file,nf1Genotype,nf2Genotype,parentId,readPair,reportMilestone,resourceType,sex,softwareLink,softwareName,species,specimenID,studyId,studyName,subtype,summary_y,tumorType_y,type
4713,Synodos NF1 Preclinical Models (Iowa Sanford E...,syn5610446,syn15672867,Active,Under Embargo,CTF,The aim of Synodos for NF1 is to find solution...,https://www.synapse.org/#!Synapse:syn5610418/w...,Jill Weimer,University of Iowa | University of Arizona | E...,Plexiform Neurofibroma,Neurofibromatosis 1,NaN,NaN,0,0,NaN,NaN,syn5610446,Synodos,NaN,1418096.0,1.453941e+12,report,NaN,Neurofibromatosis 1,NaN,70c35333-7ab2-11e9-98fa-026b0a0ad230,NaN,pdf,syn5611533,CTF,NaN,NaN,NaN,NaN,NaN,NaN,1.543522e+12,Weimer_Meyerholz_NF1 Final Application.pdf,NaN,NaN,syn5610446,NaN,NaN,report,NaN,NaN,NaN,NaN,NaN,syn5610446,Synodos NF1 Preclinical Models (Iowa Sanford E...,NaN,NaN,NaN,NaN
4714,Synodos NF1 Preclinical Models (Iowa Sanford E...,syn5610446,syn15672867,Active,Under Embargo,CTF,The aim of Synodos for NF1 is to find solution...,https://www.synapse.org/#!Synapse:syn5610418/w...,Jill Weimer,University of Iowa | University of Arizona | E...,Plexiform Neurofibroma,Neurofibromatosis 1,NaN,NaN,0,0,NaN,NaN,syn10624830,Synodos,NaN,3333546.0,1.503966e+12,NaN,NaN,Neurofibromatosis 1,NaN,8294fe8c-7ab2-11e9-98fa-026b0a0ad230,NaN,doc,syn10492340,CTF,NaN,NaN,NaN,NaN,NaN,NaN,1.543523e+12,7171 Notes.docx,NaN,NaN,syn10492244,NaN,NaN,report,NaN,NaN,NaN,Pig,7171,syn5610446,Synodos NF1 Preclinical Models (Iowa Sanford E...,NaN,NaN,NaN,NaN
4715,Synodos NF1 Preclinical Models (Iowa Sanford E...,syn5610446,syn15672867,Active,Under Embargo,CTF,The aim of Synodos for NF1 is to find solution...,https://www.synapse.org/#!Synapse:syn5610418/w...,Jill Weimer,University of Iowa | University of Arizona | E...,Plexiform Neurofibroma,Neurofibromatosis 1,NaN,NaN,0,0,NaN,NaN,syn10624830,Synodos,NaN,3333546.0,1.503966e+12,NaN,NaN,Neurofibromatosis 1,NaN,8294ff21-7ab2-11e9-98fa-026b0a0ad230,NaN,doc,syn10492341,CTF,NaN,NaN,NaN,NaN,NaN,NaN,1.543523e+12,7173 Notes.docx,NaN,NaN,syn10492245,NaN,NaN,report,NaN,NaN,NaN,Pig,7173,syn5610446,Synodos NF1 Preclinical Models (Iowa Sanford E...,NaN,NaN,NaN,NaN
4716,Synodos NF1 Preclinical Models (Iowa Sanford E...,syn5610446,syn15672867,Active,Under Embargo,CTF,The aim of Synodos for NF1 is to find solution...,https://www.synapse.org/#!Synapse:syn5610418/w...,Jill Weimer,University of Iowa | University of Arizona | E...,Plexiform Neurofibroma,Neurofibromatosis 1,NaN,NaN,0,0,NaN,NaN,syn10624830,Synodos,NaN,3333546.0,1.503966e+12,NaN,NaN,Neurofibromatosis 1,NaN,8294ffbe-7ab2-11e9-98fa-026b0a0ad230,NaN,doc,syn10492342,CTF,NaN,NaN,NaN,NaN,NaN,NaN,1.543523e+12,7174 Notes.docx,NaN,NaN,syn10492246,NaN,NaN,report,NaN,NaN,NaN,Pig,7174,syn5610446,Synodos NF1 Preclinical Models (Iowa Sanford E...,NaN,NaN,NaN,NaN
4717,Synodos NF1 Preclinical Models (Iowa Sanford E...,syn5610446,syn15672867,Active,Under Embargo,CTF,The aim of Synodos for NF1 is to find solution...,https://www.synapse.org/#!Synapse:syn5610418/w...,Jill Weimer,University of Iowa | University of Arizona | E...,Plexiform Neurofibroma,Neurofibromatosis 1,NaN,NaN,0,0,NaN,NaN,syn10624830,Synodos,NaN,3333546.0,1.503966e+12,NaN,NaN,Neurofibromatosis 1,NaN,82950056-7ab2-11e9-98fa-026b0a0ad230,NaN,doc,syn10492343,CTF,NaN,NaN,NaN,NaN,NaN,NaN,1.543523e+12,7175 Notes.docx,NaN,NaN,syn10492247,NaN,NaN,report,NaN,NaN,NaN,Pig,7175,syn5610446,Synodos NF1 Preclinical Models (Iowa Sanford E...,NaN,NaN,NaN,NaN
4718,Synodos NF1 Preclinical Models (Iowa 

In [40]:
final_df = final_df.drop(
    ["summary_x",
     "summarySource",
     "featured_x",
     "consortium_x",
     "fundingAgency_y",
     "featured_y",
     "tumorType_y",
     "etag"]
    , axis = 1)

In [41]:
final_df.columns

Index(['projectName', 'projectId', 'projectFileviewId', 'projectStatus',
       'dataStatus', 'fundingAgency_x', 'projectLeads', 'institutions',
       'tumorType_x', 'diseaseFocus', 'publication_count',
       'publication_geodata_produced', 'age', 'assay', 'benefactorId',
       'consortium_y', 'contact', 'createdBy', 'createdOn_file', 'dataSubtype',
       'dataType', 'diagnosis', 'disease', 'fileFormat', 'fileId',
       'individualID', 'isCellLine', 'isMultiIndividual', 'isMultiSpecimen',
       'link', 'manifestation', 'modifiedOn_file', 'name_file', 'nf1Genotype',
       'nf2Genotype', 'parentId', 'readPair', 'reportMilestone',
       'resourceType', 'sex', 'softwareLink', 'softwareName', 'species',
       'specimenID', 'studyId', 'studyName', 'subtype', 'summary_y', 'type'],
      dtype='object')

In [42]:
final_df.rename(columns={
    "fundingAgency_x":"fundingAgency",
    "tumorType_x":"tumorType", 
    "projectName":'name_project',
    "isCellLine":"cellLine",
    "consortium_y" : "consortium"},
                inplace=True)

In [43]:
# annotate tools files to be a resourceType tool - for now
final_df.loc[final_df.summary_y.isin(list(dfs[3].summary)), 'resourceType'] = 'tool'

In [44]:
pandas.set_option('display.max_columns', 500)
final_df.describe(include="all")

,name_project,projectId,projectFileviewId,projectStatus,dataStatus,fundingAgency,projectLeads,institutions,tumorType,diseaseFocus,publication_count,publication_geodata_produced,age,assay,benefactorId,consortium,contact,createdBy,createdOn_file,dataSubtype,dataType,diagnosis,disease,fileFormat,fileId,individualID,cellLine,isMultiIndividual,isMultiSpecimen,link,manifestation,modifiedOn_file,name_file,nf1Genotype,nf2Genotype,parentId,readPair,reportMilestone,resourceType,sex,softwareLink,softwareName,species,specimenID,studyId,studyName,subtype,summary_y,type
count,7699,7699,7699,7699,7699,7699,7699,7699,7601,7699,7699.000000,7699.0,1383,5004,7699,7241,0.0,7.699000e+03,7.699000e+03,4845,5067,6162,0.0,6760,7699,4435,4813,4713,4788,0.0,0.0,7.699000e+03,7699,4687,4693,7699,813.000000,0.0,7247,1554,0.0,0.0,4959,3687,7049,7133,0.0,0.0,0.0
unique,12,12,12,2,3,1,12,12,6,4,NaN,NaN,38,17,61,3,0.0,NaN,NaN,7,16,2,0.0,42,7699,539,2,2,2,0.0,0.0,NaN,7603,6,4,405,NaN,NaN,7,5,0.0,0.0,7,1002,20,12,0.0,0.0,0.0
top,Cutaneous Neurofibroma Data Resource,syn4984604,syn11614202,Completed,Published,CTF,Salvatore La Rosa | Annette Bakker | Justin G...,Children's Tumor Foundation | Sage Bionetworks...,Cutaneous Neurofibroma,Neurofibromatosis 1,NaN,NaN,48,rnaSeq,syn4984695,CTF Biobank,NaN,NaN,NaN,processed,geneExpression,Neurofibromatosis 1,NaN,txt,syn18879580,"N10,N5",False,False,False,NaN,NaN,NaN,User008_4PL_normalized_template_V3.1.pzfx,unknown,unknown,syn11373151,NaN,NaN,experimentalData,Female,NaN,NaN,Human,"N10,N5",syn4984604,Cutaneous Neurofibroma Data Resource,NaN,NaN,NaN
freq,3488,3488,3488,5182,4801,7699,3488,3488,3488,6011,NaN,NaN,236,1975,1623,3488,NaN,NaN,NaN,3890,1967,5410,NaN,1843,1,1297,2417,3863,3401,NaN,NaN,NaN,5,1471,2408,595,NaN,NaN,4660,1032,NaN,NaN,4484,573,3485,3485,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.642551,0.0,NaN,NaN,NaN,NaN,NaN,2.863772e+06,1.497923e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.555928e+12,NaN,NaN,NaN,NaN,1.499385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.009200,0.0,NaN,NaN,NaN,NaN,NaN,8.250526e+05,3.291304e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.476543e+09,NaN,NaN,NaN,NaN,0.500307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1.418096e+06,1.444692e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.543522e+12,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,2.166046e+06,1.473776e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.554878e+12,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,3.336397e+06,1.488988e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.560200e+12,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,0.0,NaN,NaN,NaN,NaN,NaN,3.342573e+06,1.520554e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.560201e+12,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# # double check if we didn't loose a project
if len(final_df.projectId.unique()) == len(dfs[1].projectId):
    print("All projects were successfully associated with files")
else:
    print("lost a project")

All projects were successfully associated with files


In [46]:
# check types
col_types = [col for col in list( final_df.columns ) if final_df[col].dtype == numpy.float64]
print("column names of type numpy.float64 \n:", col_types)


column names of type numpy.float64 
: ['createdBy', 'createdOn_file', 'modifiedOn_file', 'readPair', 'reportMilestone']


In [47]:
len(final_df.columns)

49

In [48]:
def changeFloatToInt(final_df, col):
    """

    :param final_df:
    :param col:
    :return:
    """
    final_df[col] = final_df[col].fillna(0).astype(int)
    final_df[col].replace(0, '', inplace=True)


In [33]:
cols = ['createdOn_file','modifiedOn_file','readPair']

[changeFloatToInt(final_df, col) for col in cols]

[None, None, None]

In [49]:
### parent is CTF data resource now
syn = synapseclient.Synapse()
syn.login()

# results = syn.tableQuery("select * from syn18956617") #current 
table = synapseclient.table.build_table("CTF Project Information Integration", 'syn4984604', final_df)





Welcome, Xengie Doan!



In [50]:
table = syn.store(table)